In [ ]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
import pandas as pd


In [ ]:
import ast

# docs = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))['data']

offline_tweets = 'Infrastructure BillSearchTerm - Infrastructure BillSearchTerm.csv' # Small data set
# offline_tweets = 'initial_quote_infrastructurebillsince202119 - initial_quote_infrastructurebillsince202119.csv' # Large data set

offline_tweets_df = pd.read_csv(offline_tweets, index_col= 0)

# Process UTF-8 encoding
offline_tweets_df['text'] = offline_tweets_df['text'].apply(lambda x: ast.literal_eval(x).decode('utf-8'))

docs = offline_tweets_df['text']

display(docs)

In [ ]:
topic_model = BERTopic(n_gram_range = (1,3), verbose=True, calculate_probabilities = True)


In [ ]:
%%time

topics, probs = topic_model.fit_transform(docs)


In [ ]:
topic_model.get_topic_info()

In [ ]:
probs.shape

In [ ]:
topic_model.get_topic(3)

In [ ]:
topic_model.get_topic_freq()

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_heatmap(n_clusters=3, top_n_topics=100)

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=40)

In [ ]:
topic_model.visualize_term_rank(log_scale=False)

In [ ]:
topic_model.visualize_barchart(top_n_topics=9, n_words=5, height=700)

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy.sparse import csr_matrix

from operator import itemgetter
import copy
import itertools

# most_valuable_edge Functions for Girvin-Newman
from networkx.algorithms.centrality import edge_betweenness_centrality

# Clustering Algorithms
from networkx.algorithms.community.asyn_fluid import asyn_fluidc
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community.kernighan_lin import kernighan_lin_bisection
from networkx.algorithms.community.modularity_max import greedy_modularity_communities
from networkx.algorithms.community.quality import modularity
from networkx.algorithms.community.quality import performance

In [ ]:
# from data

doc_id = np.array(df['paper_id'])
doc_topic = np.array(df['topic'])

# Extract array of topic probabilities, no longer needed for latest .csv layout

# topic_prob = []
# for p in df['topic_prob']:
#     row = [float(r) for r in p.strip('\n').replace(
#         '[', '').replace(']', '').split()]
#     topic_prob.append(row)

# topic_prob = np.array(topic_prob)

topic_prob = np.array(df.iloc[:, (df.columns.get_loc('topic') + 1):])

# Array of Topic labels
topic_id = ['Topic_' + str(i) for i in range(topic_prob.shape[1])]


In [ ]:
df_topic_prob = pd.DataFrame(
    data=topic_prob,
    index=np.array(doc_id),
    columns=topic_id)
df_topic_prob.insert(0, 'topic_id', doc_topic)

In [ ]:
class UndirectedDocumentGraph():
    """
    This class will be used to form various graph representations of our document corpus.
    The graph representation can be created all at once or incrementally.
    """

    def __init__(self):
        # Create an empty, undirected NetworkX graph
        self.nx_graph = nx.Graph()

    ####################
    # Graph Formation
    ####################

    def merge_graph_from_edgelist(
            self,
            path: str,
            delimiter: str = ",",
            comments: str = "#"):
        """
        Takes in an edge list from a delimited file, then unions it with current representation.
        The graphs must be disjoint.
        """
        # Load the new portion of the graph from file
        new_graph = nx.read_weighted_edgelist(
            path, delimiter=delimiter, comments=comments)

        # Merge it with the existing representation
        self.nx_graph = nx.union(new_graph, self.nx_graph)

    def merge_graph_from_sparse_scipy(self, sp_matrix):
        """
        Takes in a SciPy sparse matrix, representing our pair-wise document similarity, creates a new graph from
        it, then merges with any existing nodes.
        """
        # Load the new portion of the graph
        new_graph = nx.from_scipy_sparse_matrix(
            sp_matrix, parallel_edges=False, edge_attribute="weight")

        # An adjacency matrix will contain entries relating documents to themselves.
        # These should be removed from the graph
        new_graph.remove_edges_from(nx.selfloop_edges(new_graph))

        # Merge it with the existing representation
        self.nx_graph = nx.union(new_graph, self.nx_graph)

    def merge_graph_from_numpy_matrix(self, np_matrix):
        """
        Takes in a numpy matrix, representing our pair-wise document similarity, creates a new graph from
        it, then merges with any existing nodes.
        """
        # Load the new portion of the graph
        new_graph = nx.from_numpy_matrix(np_matrix, parallel_edges=False)

        # An adjacency matrix will contain entries relating documents to themselves.
        # These should be removed from the graph
        new_graph.remove_edges_from(nx.selfloop_edges(new_graph))

        # Merge it with the existing representation
        self.nx_graph = nx.union(new_graph, self.nx_graph)

    def merge_graph_from_pandas_df(
            self,
            pandas_df,
            source: str,
            target: str,
            edge_attr: str):
        """
        Takes in a Pandas DataFrame consisting of two columns indicating the edge connection and
        a column indicating what to use as a weight. Merges with the existing representation.
        Any filtering of edges that should not be included in the graph should be done prior to calling
        this function.
        """
        # Load the new portion of the graph
        new_graph = nx.from_pandas_edgelist(
            pandas_df, source, target, edge_attr)

        # Merge it with the existing representation
        self.nx_graph = nx.union(new_graph, self.nx_graph)

    def add_edges_with_default_weight(
            self, edge_list: list, default_weight: float):
        """
        Use this function to add edges to the graph all using some specified weight.
        edge_list is expected to be a list of tuples: [(0, 1)]  # single edge (0,1)
        ex. adding edges for citations to a graph already consisting of similarity weights
        """
        # Set the weight for all edges to the default
        edge_list_with_weight = [
            (u, v, {"weight": default_weight}) for u, v in edge_list]

        # Load the new portion of the graph
        new_graph = nx.from_edgelist(edge_list_with_weight)

        # Merge it with the existing representation
        self.nx_graph = nx.union(new_graph, self.nx_graph)

    def remove_edges_below_weight_threshold(self, threshold: float):
        """
        Removes any edges less than (inclusive) the specified threshold
        """
        edges_to_remove = [(u, v) for u, v, weight in self.nx_graph.edges.data(
            "weight") if weight <= threshold]

        self.nx_graph.remove_edges_from(edges_to_remove)

    def remove_zero_weight_edges(self):
        """
        Use this function to remove any edges that have a weight of 0, such as for documents that have zero relation.
        """
        self.remove_edges_below_weight_threshold(0)

    ####################
    # Graph Similarity
    ####################

    def get_modularity(self, graph, communities):
        """
        Calculates the modularity of a given partition of a graph. This will be one number for the whole partitioning.

        NetworkX Doc:
        https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.modularity.html#networkx.algorithms.community.quality.modularity
        """
        return modularity(graph, communities, weight="weight")

    def get_performance(self, graph, partition):
        """
        The performance of a partition is the ratio of the number of intra-community edges plus
        inter-community non-edges with the total number of potential edges

        NetworkX Doc:
        https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.quality.performance.html#networkx.algorithms.community.quality.performance
        """
        return performance(graph, partition)

    def girvan_newman(
            self,
            k: int,
            most_valuable_edge: str = "edge_betweenness_centrality"):
        """
        k - represents the number of tuples of communities from the algorithm
        most_valuable_edge - function used to get the edge removed at each iteration

        NetworkX Doc for the Girvan-Newman Method:
        https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.centrality.girvan_newman.html#networkx.algorithms.community.centrality.girvan_newman

        Helpful video explanation:
        https://youtu.be/LtQoPEKKRYM
        """
        if most_valuable_edge == "edge_betweenness_centrality_equal_weight":
            # Default option for Girvan-Newman, assumes all edges of weight 1
            comp = girvan_newman(self.nx_graph, edge_betweenness_centrality)
        elif most_valuable_edge == "edge_betweenness_centrality_equal_with_weight":
            # Take edge weight into account
            def get_edge(G):
                centrality = edge_betweenness_centrality(G, weight="weight")
                return max(centrality, key=centrality.get)
            print(get_edge(self.nx_graph))
            comp = girvan_newman(self.nx_graph, get_edge)
        elif most_valuable_edge == "least_similar":
            # Simple option of removing edge with least weight
            def get_edge(G):
                # Get edge based on the weight value (index 2 of triple)
                u, v, w = min(G.edges(data="weight"), key=itemgetter(2))
                return (u, v)
            comp = girvan_newman(self.nx_graph, get_edge)
        else:
            raise ValueError(
                "Invalid most_valuable_edge option for Girvan-Newman")

        # Create a list of dictionaries representing each row for the Pandas DF
        node_dict_list = []

        # Extract only the first specified number of communities and add them
        # to the dictionary
        num_communities = 2
        for communities in itertools.islice(comp, k):
            # Get the n number of communities
            community_tuple = tuple(sorted(c) for c in communities)

            # Calculate the modularity of the partitioning
            mod = self.get_modularity(self.nx_graph, community_tuple)

            # Calculate the performance of the partitioning
            perf = self.get_performance(self.nx_graph, community_tuple)

            # Loop through each of the communities
            for cluster_id in range(len(community_tuple)):
                # Get the list of nodes in the community
                nodes = community_tuple[cluster_id]

                # Loop through each of the nodes and form the Pandas DF row
                # dictionary
                for node in nodes:
                    row_dict = {
                        "algorithm": "Girvan-Newman",
                        "settings": "most_valuable_edge:" + most_valuable_edge,
                        "num_clusters": num_communities,
                        "cluster_id": cluster_id,
                        "node_id": node,
                        "modularity": mod,
                        "performance": perf
                    }
                    node_dict_list.append(row_dict)

            # Increment the community count
            num_communities += 1

        # Create a Pandas DF from the results
        clustering_df = pd.DataFrame(node_dict_list)

        return clustering_df

    def greedy_modularity(self, min_edge_weight: float = 0.0):
        """
        Does not take weight into account in the Modularity algorithm
        min_edge_weight - Used to remove edges below a certain threshold prior to running algorithm.

        NetworkX Doc:
        https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.modularity_max.greedy_modularity_communities.html#networkx.algorithms.community.modularity_max.greedy_modularity_communities

        Modularity on Wikipedia:
        https://en.wikipedia.org/wiki/Modularity_(networks)
        """
        # Remove edges below a certain threshold
        edges_to_remove = [(u, v) for u, v, weight in self.nx_graph.edges.data(
            "weight") if weight < min_edge_weight]
        filtered_graph = copy.deepcopy(self.nx_graph)
        filtered_graph.remove_edges_from(edges_to_remove)

        # Divide the graph into communities
        communities = greedy_modularity_communities(filtered_graph)

        # Determine how many clusters we have
        num_clusters = len(communities)

        # Calculate the modularity of the partitioning
        mod = self.get_modularity(filtered_graph, communities)

        # Calculate the performance of the partitioning
        perf = self.get_performance(filtered_graph, communities)

        # Convert into a Pandas DF
        # Create a list of dictionaries representing each row for the Pandas DF
        node_dict_list = []

        # Add a row to the DF for each individual node
        for comm, cluster_id in zip(communities, range(num_clusters)):
            # Loop through each of the nodes and form the Pandas DF row
            # dictionary
            for node in comm:
                row_dict = {
                    "algorithm": "Greedy Modularity",
                    "settings": "min_edge_weight:" + str(min_edge_weight),
                    "num_clusters": num_clusters,
                    "cluster_id": cluster_id,
                    "node_id": node,
                    "modularity": mod,
                    "performance": perf
                }
                node_dict_list.append(row_dict)

        # Create a Pandas DF from the results
        clustering_df = pd.DataFrame(node_dict_list)

        return clustering_df

    def async_fluidc(self, k: int, max_iter: int = 100):
        """
        Does not support weighted graphs
        k - number of communities to be found
        max_iter - max number of iterations if algorithm doesn't converge

        NetworkX Doc:
        https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.asyn_fluid.asyn_fluidc.html#networkx.algorithms.community.asyn_fluid.asyn_fluidc
        """
        # Find the communities based on the parameters
        communities = asyn_fluidc(self.nx_graph, k, max_iter, seed=42)

        # First need to convert iterator to list to act on it multiple times
        comm_list = list(communities)

        # Calculate the modularity of the partitioning
        mod = self.get_modularity(self.nx_graph, comm_list)

        # Calculate the performance of the partitioning
        perf = self.get_performance(self.nx_graph, comm_list)

        # Convert into a Pandas DF
        # Create a list of dictionaries representing each row for the Pandas DF
        node_dict_list = []

        # Add a row to the DF for each individual node
        for comm, cluster_id in zip(comm_list, range(k)):
            # Loop through each of the nodes and form the Pandas DF row
            # dictionary
            for node in comm:
                row_dict = {
                    "algorithm": "Fluid Communities",
                    "settings": "max_iter:" + str(max_iter),
                    "num_clusters": k,
                    "cluster_id": cluster_id,
                    "node_id": node,
                    "modularity": mod,
                    "performance": perf
                }
                node_dict_list.append(row_dict)

        # Create a Pandas DF from the results
        clustering_df = pd.DataFrame(node_dict_list)

        return clustering_df

    def kernighan_lin_bisection(self, num_cuts: int, max_iter: int = 10):
        """
        Cuts the graph into two groups at each iteration. Continues to do this recursively throughout each subgraph.
        Will stop splitting the graph at num_cuts or when a subgraph can no longer be split, whichever comes first

        num_cuts - Specifies the number of cut levels you want to make

        NetworkX Doc:
        https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.community.kernighan_lin.kernighan_lin_bisection.html#networkx.algorithms.community.kernighan_lin.kernighan_lin_bisection
        """
        # Start with the entire graph
        old_graphs = [self.nx_graph]
        new_graphs = []

        # Convert into a Pandas DF
        # Create a list of dictionaries representing each row for the Pandas DF
        node_dict_list = []

        # Perform the cuts for the specified number of levels
        for level in range(0, num_cuts):
            # Check to ensure all graphs can be split. If not, then stop
            # bisection.
            min_graph_size = min([g.number_of_nodes() for g in old_graphs])
            if min_graph_size <= 1:
                break

            # Keep track of all partitions on each level. Needed for
            # calculating modularity/performance
            level_partitions = []

            # Loop through each graph representing one of the cuts
            for g in old_graphs:
                # Split each graph into 2
                partitions = kernighan_lin_bisection(
                    g, None, max_iter=max_iter, weight="weight")

                # Track the partitions for the level
                level_partitions.extend(partitions)

                # Prepare the graphs for the next level of bisection
                for part in partitions:
                    # Get the bisected half of the graph and create a new graph of only those nodes/edges
                    # Will be used in the next iteration
                    half_graph = g.subgraph(part)
                    new_graphs.append(half_graph)

            # Calculate the modularity of the partitioning, using all graphs at
            # the level of cuts
            mod = self.get_modularity(self.nx_graph, level_partitions)

            # Calculate the performance of the partitioning
            perf = self.get_performance(self.nx_graph, level_partitions)

            # Add a row to the DF for each individual node
            num_clusters = len(level_partitions)
            for comm, cluster_id in zip(level_partitions, range(num_clusters)):
                # Loop through each of the nodes and form the Pandas DF row
                # dictionary
                for node in comm:
                    row_dict = {
                        "algorithm": "Kernighan-Lin Bisection",
                        "settings": "max_iter:" + str(max_iter),
                        "num_clusters": num_clusters,
                        "cluster_id": cluster_id,
                        "node_id": node,
                        "modularity": mod,
                        "performance": perf
                    }
                    node_dict_list.append(row_dict)

            old_graphs = new_graphs
            new_graphs = []

        # Create a Pandas DF from the results
        clustering_df = pd.DataFrame(node_dict_list)

        return clustering_df

    def undirected_pagerank(self, alpha: float = 0.85, max_iter: int = 100):
        """
        NetworkX Doc:
        https://networkx.org/documentation/stable//reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html
        """
        # Get the dictionary output from the pagerank algorithm
        pagerank_output = nx.pagerank(
            self.nx_graph,
            weight="weight",
            alpha=alpha,
            max_iter=max_iter)

        # Change the output to a Pandas DataFrame
        node_id_list = []
        page_rank_val_list = []
        for node_id, page_rank_val in pagerank_output.items():
            node_id_list.append(node_id)
            page_rank_val_list.append(page_rank_val)

        page_rank_df = pd.DataFrame.from_dict(data={
            "node_id": node_id_list,
            "page_rank_val": page_rank_val_list
        })

        # Sort by page rank value, descending
        page_rank_df.sort_values(
            by=['page_rank_val'],
            ascending=False,
            inplace=True)

        return page_rank_df

    ####################
    # Output
    ####################

    def write_edge_list(
            self,
            path: str,
            delimiter: str = ",",
            comments: str = "#"):
        """
        Writes the current contents of the UndirectedDocumentGraph object to a NetworkX edge list representation
        """
        nx.write_weighted_edgelist(
            self.nx_graph,
            path=path,
            comments=comments,
            delimiter=delimiter)

In [ ]:

# Probability threshold function


def probThreshold(data, threshold: float = 0.01):
    return np.where(data < threshold, 0, data)

# Similarity measure


def simAbsCorr(data):
    S = np.absolute(np.corrcoef(data))
    return S


def simSignedCorr(data):
    S = (1 + np.corrcoef(data)) / 2
    return S

# Adjacency functions


def powerAdj(SimMat, Beta: int = 6):
    A = SimMat ** Beta
    np.fill_diagonal(A, 0)
    return A


def signumAdj(SimMat, tau: float = 0.0):
    A = np.where(SimMat < tau, 0, 1)
    np.fill_diagonal(A, 0)
    return A

# Topological Overlap Matrix function


def TOMadjacency(AdjMat, threshold_quantile: float = 0.8):
    '''
    TOMadjacency calculates an adjacency matrix by the network overlap of nodes
    in a weighted, undirected graph.
    '''
    # Calculate common neighbors of each node
    L = AdjMat.dot(AdjMat.T)

    # Calculate connectivity of node
    Krow = AdjMat.sum(axis=1)
    Kcol = AdjMat.sum(axis=0)
    Kmin = np.array([np.minimum(k_i, Kcol) for k_i in Krow])

    # Topological overlap
    TOM = (L + AdjMat) / (Kmin + 1 - AdjMat)

    TOM_filtered = np.where(
        TOM >= np.quantile(
            TOM, threshold_quantile), TOM, 0)

    np.fill_diagonal(TOM_filtered, 0)

    TOMlower = np.tril(TOM_filtered)

    TOMsparse = csr_matrix(TOMlower)

    return TOMsparse


In [ ]:
thresh_val = 1 / topic_prob.shape[1]

topic_prob_sigProbs = probThreshold(topic_prob, threshold=thresh_val)

zeroTopic_doc = np.where(topic_prob_sigProbs.sum(axis=1) == 0)[0].tolist()

doc_kept = np.delete(doc_id, zeroTopic_doc)

topic_prob_filtered = np.delete(topic_prob_sigProbs, zeroTopic_doc, axis=0)

doc_topic_kept = np.delete(doc_topic, zeroTopic_doc)

In [ ]:
S_topic = simSignedCorr(topic_prob_filtered.T)

A_topic = signumAdj(S_topic, tau=np.quantile(S_topic, 0.8))

TOM_topic = TOMadjacency(A_topic, threshold_quantile=0.9)


In [ ]:
topic_graph = UndirectedDocumentGraph()

topic_graph.merge_graph_from_sparse_scipy(TOM_topic)

# Assign graph node names to topic id's
topic_label_mapping = dict(zip(topic_graph.nx_graph, topic_id))
topic_graph.nx_graph = nx.relabel_nodes(
    topic_graph.nx_graph, topic_label_mapping)

In [ ]:
my_graph = topic_graph

output_dfs = []
diagnostic_dfs = []

print("Girvan-Newman (edge_betweenness_centrality_equal_with_weight):")
girvan_newman_df_ebcw = my_graph.girvan_newman(
    20, "edge_betweenness_centrality_equal_with_weight")
output_dfs.append(girvan_newman_df_ebcw)
girvan_newman_df_ebcw_diag = girvan_newman_df_ebcw[[
    'num_clusters', 'modularity', 'performance']].drop_duplicates(ignore_index=True)
diagnostic_dfs.append(girvan_newman_df_ebcw_diag)
print(girvan_newman_df_ebcw_diag, "\n")